## 1. Importing data and data preprocessing

### 1.1 Import necesary libraries

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import json
import random
import tensorflowjs as tfjs

### 1.2 Load and preprocess data

In [2]:
# Prepare training data
training_data = []

# Hold training
with open('hold_training.json') as json_file:  
    data = json.load(json_file)
    for coordSet in data['coords']:
        # Separate x and y coords
        training_data.append([[coordSet['mousePos'][0::2],coordSet['mousePos'][1::2]],0]) # 0 is hold
        
# Spill training
with open('spill_training.json') as json_file:  
    data = json.load(json_file)
    for coordSet in data['coords']:
        training_data.append([[coordSet['mousePos'][0::2],coordSet['mousePos'][1::2]],1]) # 1 is spill

# Shuffling
random.shuffle(training_data)

# Extract
train_coords = []
train_labels = []
minValX = float('inf')
maxValX = 0
minValY = float('inf')
maxValY = 0
for sample in training_data:
    train_coords.append(sample[0])
    train_labels.append(sample[1])
    minValX = min(minValX, min(sample[0][0]))
    maxValX = max(maxValX, max(sample[0][0]))
    minValY = min(minValY, min(sample[0][1]))
    maxValY = max(maxValY, max(sample[0][1]))
# Normalize
difValX = maxValX - minValX
difValY = maxValY - minValY
for i in range(len(train_coords)):
    for j in range(len(train_coords[i][0])):
        train_coords[i][0][j] = (train_coords[i][0][j] - minValX) / float(difValX)
        train_coords[i][1][j] = (train_coords[i][1][j] - minValY) / float(difValY)
        
        train_coords[i][0] = np.asarray(train_coords[i][0])
        train_coords[i][1] = np.asarray(train_coords[i][1])
    # Convert to np array
    train_coords[i] = np.asarray(train_coords[i])

# Convert
train_coords = np.asarray(train_coords)
train_labels = np.asarray(train_labels)
print(train_coords)
print(train_labels)

[[[0.18452861 0.18452861 0.18452861 ... 0.43916197 0.43916197 0.44238517]
  [0.19433962 0.19433962 0.19433962 ... 0.19245283 0.19245283 0.19245283]]

 [[0.19178082 0.19178082 0.19178082 ... 0.43835616 0.44319098 0.44319098]
  [0.01320755 0.01320755 0.01320755 ... 0.15283019 0.15660377 0.15660377]]

 [[0.46494762 0.46494762 0.46494762 ... 0.46414182 0.46414182 0.46414182]
  [0.81698113 0.81698113 0.81698113 ... 0.33773585 0.33207547 0.33207547]]

 ...

 [[0.66720387 0.66720387 0.66720387 ... 0.53585818 0.53021757 0.53021757]
  [0.44150943 0.44150943 0.44150943 ... 0.30566038 0.3        0.3       ]]

 [[0.08783239 0.08783239 0.08783239 ... 0.55922643 0.55922643 0.51087832]
  [0.11698113 0.11698113 0.11698113 ... 0.38113208 0.38113208 0.33018868]]

 [[0.66800967 0.66800967 0.66720387 ... 0.50443191 0.5004029  0.5004029 ]
  [0.09811321 0.09811321 0.1        ... 0.1490566  0.1490566  0.1490566 ]]]
[0 0 0 0 0 1 0 1 1 0 1 1 1 1 1 1 0 0 1 0]


In [3]:
# Prepare testing data
testing_data = []

# Hold testing
with open('hold_testing.json') as json_file:  
    data = json.load(json_file)
    for coordSet in data['coords']:
        # Separate x and y coords
        testing_data.append([[coordSet['mousePos'][0::2],coordSet['mousePos'][1::2]],0]) # 0 is hold
        
# Spill testing
with open('spill_testing.json') as json_file:  
    data = json.load(json_file)
    for coordSet in data['coords']:
        testing_data.append([[coordSet['mousePos'][0::2],coordSet['mousePos'][1::2]],1]) # 1 is spill

# Shuffling
random.shuffle(testing_data)

# Extract
test_coords = []
test_labels = []
minValX = float('inf')
maxValX = 0
minValY = float('inf')
maxValY = 0
for sample in testing_data:
    test_coords.append(sample[0])
    test_labels.append(sample[1])
    minValX = min(minValX, min(sample[0][0]))
    maxValX = max(maxValX, max(sample[0][0]))
    minValY = min(minValY, min(sample[0][1]))
    maxValY = max(maxValY, max(sample[0][1]))
# Normalize
difValX = maxValX - minValX
difValY = maxValY - minValY
for i in range(len(test_coords)):
    for j in range(len(test_coords[i][0])):
        test_coords[i][0][j] = (test_coords[i][0][j] - minValX) / float(difValX)
        test_coords[i][1][j] = (test_coords[i][1][j] - minValY) / float(difValY)
        
        test_coords[i][0] = np.asarray(test_coords[i][0])
        test_coords[i][1] = np.asarray(test_coords[i][1])
    # Convert to np array
    test_coords[i] = np.asarray(test_coords[i])

# Convert
test_coords = np.asarray(test_coords)
test_labels = np.asarray(test_labels)
print(test_coords.shape)
print(len(test_labels))

(20, 2, 150)
20


## 2. CNN

### 2.1 Build model

In [4]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(2,150)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

### 2.2 Compile 

In [5]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

### 2.3 Train

In [6]:
model.fit(train_coords, train_labels, epochs=50)

W0717 15:53:53.041600 4521743808 deprecation.py:323] From /Users/tgoh/Documents/sippy-cup/env/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/50
20/20 [==============================] - 0s 3ms/sample - loss: 0.8531 - acc: 0.5000
Epoch 2/50
20/20 [==============================] - 0s 107us/sample - loss: 0.6994 - acc: 0.5500
Epoch 3/50
20/20 [==============================] - 0s 81us/sample - loss: 0.6935 - acc: 0.4000
Epoch 4/50
20/20 [==============================] - 0s 99us/sample - loss: 0.6998 - acc: 0.5000
Epoch 5/50
20/20 [==============================] - 0s 98us/sample - loss: 0.6711 - acc: 0.5000
Epoch 6/50
20/20 [==============================] - 0s 101us/sample - loss: 0.6282 - acc: 0.6000
Epoch 7/50
20/20 [==============================] - 0s 70us/sample - loss: 0.5947 - acc: 0.7500
Epoch 8/50
20/20 [==============================] - 0s 59us/sample - loss: 0.5811 - acc: 0.8000
Epoch 9/50
20/20 [==============================] - 0s 61us/sample - loss: 0.5779 - acc: 0.8000
Epoch 10/50
20/20 [==============================] - 0s 60us/sample - loss: 0.5718 - acc: 0.8000
Epoch 11/50
20/20 [===================

### 2.4 Results

In [7]:
test_loss, test_acc = model.evaluate(test_coords, test_labels)

20/20 [==============================] - 0s 856us/sample - loss: 0.4722 - acc: 0.8000


In [8]:
predictions = model.predict(test_coords)
print(predictions)

[[0.08783162]
 [0.21434844]
 [0.8796797 ]
 [0.6541165 ]
 [0.9894339 ]
 [0.39679602]
 [0.59016925]
 [0.40661946]
 [0.1633619 ]
 [0.5297847 ]
 [0.58492196]
 [0.20273101]
 [0.22926798]
 [0.37387815]
 [0.45531476]
 [0.55732757]
 [0.08413476]
 [0.4441853 ]
 [0.71137273]
 [0.15819786]]


### 2.5 Save model

In [9]:
# Python model
model.save('Model/model.h5')
# JS model
tfjs.converters.save_keras_model(model, './ModelJS')